## Covariates Generation Testing

In [1]:
# import the library
import ee
import geemap
from geemap import ee_export_image_collection_to_drive
import numpy as np
import matplotlib as plt
import pandas as pd
import matplotlib.pyplot as plt
from optical_cov import get_aoi_from_gaul,  get_optical_data, get_temporal_composite, apply_brdf, terrain_correction, cal_indices
from radar_cov import get_palsar_data, radar_annual_comp, get_s1_data, s1_seasonal_composite
import anci_cov
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

Number of images in collection:  13
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed


This message:
Number of images in collection:  13
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed
came from the S1-ard functions, not yet resolve

## Select and Visualize the AOI

In [2]:
# --- Area of Interest (AOI) ---
# Set the country and province for the AOI using GAUL admin boundaries
COUNTRY = "Indonesia"
PROVINCE = "Sumatera Selatan"
aoi = get_aoi_from_gaul(country=COUNTRY, province=PROVINCE)
# Visualise the AOI
Map = geemap.Map() 
Map.addLayer(aoi, 
            {'color': 'red', 'fillColor': '00000000'}, 
            'Area of Interest (AOI)',)
# Visualize AOI
Map.centerObject(aoi, 8)
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## Optical Remote Sensing Data Retrival

In [11]:
#optical data type as follows:
#L8_TOA: Landsat 8 Collection 2 Top of Atmosphere 
#L8_SR: Landsat 8 Collection 2 Surface Reflectance
#S2_TOA: Sentinel-2 L1C Top of Atmosphere
#S2_SR: Sentienl-2 L2A Surface Reflectance
start = '2017-01-01'
end = '2018-01-01'
l8_toa = get_optical_data(aoi, start_date=start, end_date=end, cloud_cover=40,
                          optical_data='L8_TOA')
l8_sr = get_optical_data(aoi, start_date=start, end_date=end,
                          optical_data='L8_SR')
l8_toa_visparam = {
    min: 0,
    max: 0.5,
    'bands':['red', 'green', 'blue']
}
l8_sr_visparam = {
    min: 0,
    max: 0.3,
    'bands':['red', 'green', 'blue']
}
# Adding Basemap
m = geemap.Map()
# set center of the map in the area of interest
m.centerObject(aoi, 7)
# adding the data
m.addLayer(l8_toa, l8_toa_visparam, 'Landsat 8 TOA')
m.addLayer(l8_sr, l8_sr_visparam, 'Landsat 8 SR')
m


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## BDRF and Topographic Correction 

In [5]:
#BRDF Correction
bdrf_toa= l8_toa.map(apply_brdf)
m.addLayer(bdrf_toa, l8_sr_visparam, 'Landsat 8 TOA_BRDF')
print(print(bdrf_toa.first().bandNames().getInfo()))
#Terrain Correction
bdrf_terrain_cor = terrain_correction(bdrf_toa)
m.addLayer(bdrf_terrain_cor, l8_sr_visparam, 'Landsat 8 TOA_BRDF_Topo')
m
#The result is not yet compared. Visual representation have minimal difference

['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
None


Map(bottom=16986.0, center=[-3.3182087091819388, 104.34041214314226], controls=(WidgetControl(options=['positi…

## Temporal Composite

In [13]:
#Example for median composite
l8_corrected_median = get_temporal_composite(bdrf_terrain_cor, method='median', aoi=aoi)
#Example for 25th percentile
l8_corrected_p25 = get_temporal_composite(bdrf_terrain_cor, method= 'Percentile', aoi = aoi, percentile_value=25)
m.addLayer(l8_corrected_median, l8_toa_visparam, 'Landsat Median Composite')
m.addLayer(l8_corrected_p25,     
           {min: 0,max: 0.3,
           'bands':['red_p25', 'green_p25', 'blue_p25']}, '25th Percentile L8 TOA')
#Note: Restore+ framework applied different reducer for different bands, not yet implemented here
m

Map(bottom=16977.0, center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['positi…

## Spectral Transformation

In [16]:
#calculate all indices
l8_indices =bdrf_terrain_cor.map(cal_indices)
#Select NDVI
ndvi = l8_indices.select('NDVI')
m.addLayer(ndvi, {min: 0, max: 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI_Corrected')
m

Map(bottom=16977.0, center=[-3.2173020581871374, 104.16137695312501], controls=(WidgetControl(options=['positi…

## Get Radar Remote Sensing Data
### ALOS PALSAR ScanSAR 2

In [20]:
#ALOS PALSAR 2017 (output in backscatter coefficient gamma nought)
palsar_2017 = get_palsar_data(aoi, start_date =start, end_date = end, polarizations=['HH'])
#Radar Annual Composite
median_palsar = radar_annual_comp(aoi, palsar_2017, reducer='median')
map = geemap.Map()
map.centerObject(aoi, 7)
map.addLayer(palsar_2017.select(['HH']), {}, 'HH polarization')
map.addLayer(median_palsar.clip(aoi), {}, 'Median HH polarization')
map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### Sentinel-1 GRD

In [22]:
#Called the collection and preprocessed them using s1_ard 
s1_data = get_s1_data(aoi, start, end, clip_to_aoi = True)
#Sentinel-1 Annual composite
median_s1 = radar_annual_comp(aoi, s1_data, reducer='median')
map.addLayer(s1_data, {}, 'Sentinel-1_prep')
map.addLayer(median_s1.clip(aoi), {}, 'Median Sentinel-1_prep')
map
#Note: the result is not yet shown since using public GEE acount resulted in user memory limit

Number of images in collection:  216
Additional border noise correction is completed
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed


Map(bottom=16977.0, center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['positi…

## Sentinel-1 Seasonal Composite

In [ ]:
#Sentinel-1 Seasonal Composite, while using precipitation data from CHIRPS
s1_seasonal = s1_seasonal_composite(s1_data, aoi, start, end,
                        precip_threshold = 170, #Example value for Sumatera Selatan (in mm)
                           rainy_month = None, #specified rainy months if CHIRPS data is not used
                           dry_month = None, #specified dry months if CHIRPS data is not used
                           reducer = 'median'
                           )


Month 1: Avg Rainfall = 340.97 mm
Month 2: Avg Rainfall = 284.20 mm
Month 3: Avg Rainfall = 252.61 mm
Month 4: Avg Rainfall = 344.80 mm
Month 5: Avg Rainfall = 281.12 mm
Month 6: Avg Rainfall = 134.33 mm
Month 7: Avg Rainfall = 165.57 mm
Month 8: Avg Rainfall = 159.70 mm
Month 9: Avg Rainfall = 164.63 mm
Month 10: Avg Rainfall = 275.18 mm
Month 11: Avg Rainfall = 333.08 mm
Month 12: Avg Rainfall = 342.40 mm


In [26]:
m.addLayer(s1_seasonal, {}, 'S1 Seasonal Composite')
m

Map(bottom=16977.0, center=[-3.2173020581871374, 104.16137695312501], controls=(WidgetControl(options=['positi…